In [1]:
from dms_stan.datasets.trpb import (
    TrpBExponentialGrowth,
    TrpBSigmoidGrowthInitParam,
    TrpBSigmoidGrowth,
)

SOURCE_FILE = "~/GitRepos/DMSStan/raw_data/trpb/3-site_merged_replicates/LibI/20230926/LibI_merged_AAs.csv"

Prior predictive check for the TrpB exponential growth model:

In [ ]:
EXP_MODEL = TrpBExponentialGrowth.from_data_file(SOURCE_FILE)
EXP_MODEL.prior_predictive()

BokehModel(combine_events=True, render_bundle={'docs_json': {'ab2d6fbb-9e89-47d2-8210-8e3d946f0866': {'version…

Now a slightly more expressive model: Sigmoid growth parametrized using initial abundances:

In [ ]:
SIG_INIT_MODEL = TrpBSigmoidGrowthInitParam.from_data_file(SOURCE_FILE)
SIG_INIT_MODEL.prior_predictive()

BokehModel(combine_events=True, render_bundle={'docs_json': {'a72f1f77-2451-41f4-b52a-a25b74cb29d1': {'version…

Slightly more expressive again: Sigmoid growth with variable growth rates and inflection points, but assuming identical maximum abundances for all variants.

In [ ]:
SIG_MODEL = TrpBSigmoidGrowth.from_data_file(SOURCE_FILE)
SIG_MODEL.prior_predictive()

BokehModel(combine_events=True, render_bundle={'docs_json': {'1fb71d71-f110-442c-95db-ea5641d828d6': {'version…

# MAP

Now that we've selected our priors, we're ready to identify the MAP for each.

In [ ]:
# EXP_MAP = EXP_MODEL.approximate_map(early_stop=10, device=0, seed=1025)
# EXP_MAP.plot_loss_curve()

Epochs:  21%|██        | 20698/100000 [04:36<17:39, 74.84it/s, -log pdf/pmf=1827153.22]


:Curve   [iteration]   (-log pdf/pmf)

We can plot the posterior predictive checks for the MAP:

In [ ]:
# EXP_MAP.get_inference_obj(batch_size=50).run_ppc(logy_ppc_samples=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'71b889fe-9486-4b3b-9359-fa3a9019ae3f': {'version…

Same for the abundance-initialized sigmoid model:

In [ ]:
# SIG_INIT_MAP = SIG_INIT_MODEL.approximate_map(early_stop=10, device=0, seed=1025)
# SIG_INIT_MAP.plot_loss_curve()

Epochs:  17%|█▋        | 17479/100000 [04:39<21:58, 62.58it/s, -log pdf/pmf=604559.78] 


:Curve   [iteration]   (-log pdf/pmf)

In [ ]:
# SIG_INIT_INF_OBJ = SIG_INIT_MAP.get_inference_obj(batch_size=50)
# SIG_INIT_INF_OBJ.run_ppc(logy_ppc_samples=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'f3134c8f-4d43-438b-a950-dd0ba13a5c4c': {'version…

And for the full sigmoid model:

In [ ]:
# SIG_MAP = SIG_MODEL.approximate_map(early_stop=10, device=0, seed=1025)
# SIG_MAP.plot_loss_curve()

Epochs:  17%|█▋        | 16805/100000 [04:35<22:45, 60.91it/s, -log pdf/pmf=464073.53] 


:Curve   [iteration]   (-log pdf/pmf)

In [ ]:
# SIG_MAP.get_inference_obj(batch_size=50).run_ppc(logy_ppc_samples=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'c080728b-9dd5-4f2f-bf0e-7fe5b39e1aef': {'version…

# MCMC

Finally, we will use Stan to sample from the posterior. Sampling is likely to take some time with these models, so we're going to compile an object that will allow us to run sampling outside of the notebook:

In [ ]:
EXP_MODEL.mcmc(
    output_dir="./exponential",
    cpp_options={"STAN_THREADS": True},
    seed=1025,
    delay_run=True,
    iter_warmup=2000,
    threads_per_chain=4,
)
SIG_INIT_MODEL.mcmc(
    output_dir="./sigmoid_init",
    cpp_options={"STAN_THREADS": True},
    seed=1025,
    delay_run=True,
    iter_warmup=2000,
    threads_per_chain=4,
)
SIG_MODEL.mcmc(
    output_dir="./sigmoid",
    cpp_options={"STAN_THREADS": True},
    seed=1025,
    delay_run=True,
    iter_warmup=2000,
    threads_per_chain=4,
)

AttributeError: Attribute '_shared_alpha' not found in Dirichlet

Now run analysis on the diagnostics and report:

In [ ]:
# samples = SampleResults.from_disk("/home/bwittmann/GitRepos/DMSStan/flip3/trpB/sigmoid/model-20250410192656-20250410192733_arviz.nc", skip_fit=True)

In [ ]:
# _ = samples.diagnose()

In [ ]:
# samples.run_ppc(logy_ppc_samples=True)

In [ ]:
# samples.inference_obj